In [79]:
import requests
import pandas as pd
import io
import os
from tabulate import tabulate
from dotenv import load_dotenv
from IPython.display import HTML

In [80]:
def get_stock_list():
    url = 'https://www.alphavantage.co/query?function=LISTING_STATUS&apikey=' + str(os.environ.get('API_KEY'))
    r = requests.get(url)
    data = pd.read_csv(io.StringIO(r.content.decode('utf-8')))
    return list(data['symbol'])


def get_stock_df(ticker):
    url = 'https://www.alphavantage.co/query?function=TIME_SERIES_DAILY&symbol=' + ticker + '&apikey=' + str(os.environ.get('API_KEY'))
    r = requests.get(url)
    data = r.json()

    df = pd.DataFrame.from_dict(data, orient='columns')

    df = df.drop("Meta Data", axis = 1)
    df = df.drop(index=["1. Information","2. Symbol","3. Last Refreshed","4. Output Size","5. Time Zone"])

    HTML(tabulate(df, headers='keys', tablefmt='html'))


    df_normalized = pd.json_normalize(df['Time Series (Daily)'])
    df = df.reset_index()
    df_final = df.join(df_normalized)
    df_final.drop('Time Series (Daily)', axis = 1, inplace = True)
    df_final.index = df['index']
    df_final.drop('index', axis = 1, inplace = True)
    return df_final

In [85]:
####　EXAMPLE 

stock_list = get_stock_list()

for stock in stock_list[:5]:
    df = get_stock_df(stock)
    df.to_pickle("../stock_data/" + stock + '.pkl')